# Skript for TMS Data Cleaning - Part 2

<strong><em>Important: This is a guide, which helps and explains you the data cleaning we where doing before this Hack-a-thon. There are parts you can and sometimes should directly copy and paste. You won't be able to copy the whole notebook and run it within your project.</em></strong>

## Creating the Client Connection to the Cloud Object Storage and the "smart-city-live-vehicle-positions" bucket

The following code cell can be automatically inserted trough the Notebook UI. To do so, click on the data button (top right corner) there you find the *files* and *connections* tab. Go to the *connection* as we want to create a client to our Cloud Object Storage. 

There you will find the Connection which we created before. Click "insert to code" and choose the "StreamingBody object" option. After that there will open a pop up which showes you the folder structure of your underlying cloud bucket. Choose the right folders and subfolders until you end up in the last subfolder, that contains all the .json files we need. Choose one file and click *Select*. Next you will see a code cell, inserted automatically, that looks like this one except it contains the correct api-keys etc.

> It doesn't matter which .json you will choose, because we will later on only use the created client object to access more then only one .json file.

## Converting the tst into a other timestamp format 

We want to do this, because
1. SPSS Modeller don't understand the given format in `tst`
2. We maybe want to enrich the data by JOIN it with another resource.
   
So we define a method, which takes every `tst` value and save a changed version of that into a list, which becomes a new column after that.


In [8]:
import datetime

date = []

for x in df_tms.measuredTime:
    x = x[:-7]
    date_obj = datetime.datetime.strptime(x, '%Y-%m-%dT%H')
    date.append(str(date_obj.date()) + " " + str(date_obj.time()))


df_tms["timestamp"] = date

In [9]:
df_tms.head()

,index,measuredTime,roadStationId,oldName,name,sensorUnit,id,shortName,sensorValue,timeWindowStart,timeWindowEnd,lastUpdated,lastError,type,status,timestamp
0,0,2022-03-12T23:59:35Z,23575.0,keskinopeus_5min_liukuva_suunta1_VVAPAAS1,KESKINOPEUS_5MIN_LIUKUVA_SUUNTA1_VVAPAAS1,***,5158.0,LTila1,139.0,NaN,NaN,NaN,NaN,NaN,NaN,2022-03-12 23:00:00
1,1,2022-03-12T23:59:35Z,23575.0,ohitukset_5min_liukuva_suunta1_MS1,OHITUKSET_5MIN_LIUKUVA_SUUNTA1_MS1,***,5164.0,MTila1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,2022-03-12 23:00:00
2,2,2022-03-12T23:59:35Z,23575.0,ohitukset_60min_kiintea_suunta2_MS2,OHITUKSET_60MIN_KIINTEA_SUUNTA2_MS2,***,5071.0,MTil2,1.0,2022-03-12T22:00:00Z,2022-03-12T23:00:00Z,NaN,NaN,NaN,NaN,2022-03-12 23:00:00
3,3,2022-03-12T23:59:35Z,23575.0,keskinopeus_60min_kiintea_suunta2,KESKINOPEUS_60MIN_KIINTEA_SUUNTA2,km/h,5057.0,km/h2,103.0,2022-03-12T22:00:00Z,2022-03-12T23:00:00Z,NaN,NaN,NaN,NaN,2022-03-12 23:00:00
4,4,2022-03-12T23:59:35Z,23575.0,ohitukset_5min_liukuva_suunta2_MS2,OHITUKSET_5MIN_LIUKUVA_SUUNTA2_MS2,***,5168.0,MTila2,1.0,NaN,NaN,NaN,NaN,NaN,NaN,2022-03-12 23:00:00


## First "pre" cleaning step; drop duplicates `roadStationId`, `id` and `timestamp` 

We had to do this action at this point of the process, because we figured out over the time, that there are many duplicates in this data. Which significantly slowed down the following calculations. 

So, by using these three attributes we kind of build a __"primary key"__ to make sure, only duplicates are dropped.

In [10]:
df_tms = df_tms.drop_duplicates(subset=['roadStationId', 'id', 'timestamp'])

Just compare the shape from before (~row 7) and now. We were able to clean a lot of duplicates from the DataFrame.

In [11]:
df_tms.shape

(18924, 16)

In [12]:
df_tms.head()

,index,measuredTime,roadStationId,oldName,name,sensorUnit,id,shortName,sensorValue,timeWindowStart,timeWindowEnd,lastUpdated,lastError,type,status,timestamp
0,0,2022-03-12T23:59:35Z,23575.0,keskinopeus_5min_liukuva_suunta1_VVAPAAS1,KESKINOPEUS_5MIN_LIUKUVA_SUUNTA1_VVAPAAS1,***,5158.0,LTila1,139.0,NaN,NaN,NaN,NaN,NaN,NaN,2022-03-12 23:00:00
1,1,2022-03-12T23:59:35Z,23575.0,ohitukset_5min_liukuva_suunta1_MS1,OHITUKSET_5MIN_LIUKUVA_SUUNTA1_MS1,***,5164.0,MTila1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,2022-03-12 23:00:00
2,2,2022-03-12T23:59:35Z,23575.0,ohitukset_60min_kiintea_suunta2_MS2,OHITUKSET_60MIN_KIINTEA_SUUNTA2_MS2,***,5071.0,MTil2,1.0,2022-03-12T22:00:00Z,2022-03-12T23:00:00Z,NaN,NaN,NaN,NaN,2022-03-12 23:00:00
3,3,2022-03-12T23:59:35Z,23575.0,keskinopeus_60min_kiintea_suunta2,KESKINOPEUS_60MIN_KIINTEA_SUUNTA2,km/h,5057.0,km/h2,103.0,2022-03-12T22:00:00Z,2022-03-12T23:00:00Z,NaN,NaN,NaN,NaN,2022-03-12 23:00:00
4,4,2022-03-12T23:59:35Z,23575.0,ohitukset_5min_liukuva_suunta2_MS2,OHITUKSET_5MIN_LIUKUVA_SUUNTA2_MS2,***,5168.0,MTila2,1.0,NaN,NaN,NaN,NaN,NaN,NaN,2022-03-12 23:00:00


# Loading metadata

By now TMS comes with the value `roadStationId`, which don't provides us directly with a given position in Finland. To overcome that missing information, the TMS-API also provides us with metadata information about the `roadStationId`s. 
These metadata informations are also written into the same object storage, within a different bucket (`smart-city-tms-stations-metadata`). The value of `metastarttime` has been set beforehand and is choosen to be larger than the starttime, because it can occure that TMS data about certain `roadStationId`s is available, but there is no metadata available within the same timewindow. So we choose a bigger one, to cover all the possible `roadStaionId`s. 

In [13]:
#metadaten der TMS Stations
objs_tms_meta=get_s3_objects(s3=connection_TMS_trafficData_client,bucket="smart-city-tms-stations-metadata", last_modified_min=metastarttime, last_modified_max=endtime)
df_tms_meta = pd.DataFrame()

for obj in objs_tms_meta:
    if df_tms_meta.empty:
        df_tms_meta = pd.read_json(connection_TMS_trafficData_client.get_object(Bucket='smart-city-tms-stations-metadata', Key=obj['Key'])['Body'].read(), lines=True)
    else:
        df_tms_meta_tmp = pd.read_json(connection_TMS_trafficData_client.get_object(Bucket='smart-city-tms-stations-metadata', Key=obj['Key'])['Body'].read(), lines=True)
        df_tms_meta = df_tms_meta.append(df_tms_meta_tmp)
df_tms_meta = df_tms_meta.reset_index()

## Transform metadata

In the `df_tms_meta` there are many informations which we don't care about, we only want to access the geometry data. after that, we collect the columns `['longitude', 'latitude']` to start building our DataFrame (`df`). Then we take another (support) DataFrame, which only contains the various id's. We reset their indexes, JOIN them and drop all the duplicated `id`s aka `roadStationId`s and empty rows. 

In [ ]:
#Datentransformation
df = pd.json_normalize(df_tms_meta['geometry'])
print(df)
df = pd.DataFrame(df.iloc[:,0].tolist(), columns=['longitude', 'latitude', 'else'])
df = df[['latitude', 'longitude']]
df2 = df_tms_meta['id']
df2 = df2.reset_index()
df = df.reset_index()
df = df.join(df2.set_index('index'), on='index', how='inner')
df = df[['latitude', 'longitude', 'id']]
df = df.rename(columns={'id': 'roadStationId'})
df = df.drop_duplicates(subset=['roadStationId'])
df.dropna()

# Join metadata with tms data

Now we have two dataframes: `tms` which does contain all the sensor values and `metadata` that contains the geolocation of every `roadStationId`. 

So the next steps brings these two DataFrames together (pd.merge()). 

In [ ]:
df_tms = pd.merge(df_tms, df, left_on='roadStationId', right_on='roadStationId', how='left')

We make sure, that no row went missing and we appended our dataframe with 2 columns `[latitude, longitude]`

In [16]:
df_tms.shape

(18924, 18)

In [17]:
df_tms.head()

,index,measuredTime,roadStationId,oldName,name,sensorUnit,id,shortName,sensorValue,timeWindowStart,timeWindowEnd,lastUpdated,lastError,type,status,timestamp,latitude,longitude
0,0,2022-03-12T23:59:35Z,23575.0,keskinopeus_5min_liukuva_suunta1_VVAPAAS1,KESKINOPEUS_5MIN_LIUKUVA_SUUNTA1_VVAPAAS1,***,5158.0,LTila1,139.0,NaN,NaN,NaN,NaN,NaN,NaN,2022-03-12 23:00:00,60.486397,26.54646
1,1,2022-03-12T23:59:35Z,23575.0,ohitukset_5min_liukuva_suunta1_MS1,OHITUKSET_5MIN_LIUKUVA_SUUNTA1_MS1,***,5164.0,MTila1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,2022-03-12 23:00:00,60.486397,26.54646
2,2,2022-03-12T23:59:35Z,23575.0,ohitukset_60min_kiintea_suunta2_MS2,OHITUKSET_60MIN_KIINTEA_SUUNTA2_MS2,***,5071.0,MTil2,1.0,2022-03-12T22:00:00Z,2022-03-12T23:00:00Z,NaN,NaN,NaN,NaN,2022-03-12 23:00:00,60.486397,26.54646
3,3,2022-03-12T23:59:35Z,23575.0,keskinopeus_60min_kiintea_suunta2,KESKINOPEUS_60MIN_KIINTEA_SUUNTA2,km/h,5057.0,km/h2,103.0,2022-03-12T22:00:00Z,2022-03-12T23:00:00Z,NaN,NaN,NaN,NaN,2022-03-12 23:00:00,60.486397,26.54646
4,4,2022-03-12T23:59:35Z,23575.0,ohitukset_5min_liukuva_suunta2_MS2,OHITUKSET_5MIN_LIUKUVA_SUUNTA2_MS2,***,5168.0,MTila2,1.0,NaN,NaN,NaN,NaN,NaN,NaN,2022-03-12 23:00:00,60.486397,26.54646
